In [328]:
import pandas as pd
import requests
import numpy as np
import os 
import matplotlib.pyplot as plt
import unidecode
from difflib import SequenceMatcher
from highlight_text import HighlightText, ax_text, fig_text
from PIL import Image
import matplotlib.lines as lines
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.gridspec as gridspec

In [9]:
spi = pd.read_html('https://projects.fivethirtyeight.com/soccer-predictions/global-club-rankings/')[0]
spi.columns = spi.columns.droplevel()
spi = spi[['team','League','League country','off.','def.','spi']]

In [18]:
epl = spi[(spi['League']=='Premier League' )& (spi['League country']=='England')].reset_index(drop=True)

In [21]:
avg_def = spi['def.'].mean()
avg_off = spi['off.'].mean()
epl_def = epl['def.'].mean()
epl_off = epl['off.'].mean()

In [30]:
off_adj = epl_def/avg_def 
def_adj = epl_off/avg_off 

In [31]:
#the average epl team will concede 50% less goals = the averaege epl team will score 50% less goals 
#the average epl team will score 165% more goals = the averaege epl team will concede 165% more goals 

In [32]:
#adjustment 
epl['off_adj'] = epl['off.'] * off_adj
epl['def_adj'] = epl['def.'] * def_adj

In [41]:
base_url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r= requests.get(base_url).json()


In [49]:
id_map = {x.get('id'):x.get('name') for x in r.get('teams')}

In [50]:
base_url = 'https://fantasy.premierleague.com/api/fixtures/'
r= requests.get(base_url).json()


In [262]:
df = pd.DataFrame(r)[['event','kickoff_time','team_a','team_h']]

In [263]:
df['team_a'] = df['team_a'].map(id_map)
df['team_h'] = df['team_h'].map(id_map)

In [264]:
df['team_a'].sort_values().unique().tolist()
fpl_map = dict(zip(df['team_a'].sort_values().unique().tolist(),epl['team'].sort_values().tolist()))

In [265]:
df['team_a'] = df['team_a'].map(fpl_map)
df['team_h'] = df['team_h'].map(fpl_map)

In [266]:
epl['off rank'] = epl['off.'].rank()
epl['def rank'] = epl['def.'].rank()

In [267]:
df = df.merge(epl[['team','off rank','def rank']],how='left',left_on='team_a',right_on='team').merge(epl[['team','off rank','def rank']],how='left',left_on='team_h',right_on='team')

In [268]:
df.drop(['team_x','team_y'],axis=1,inplace=True)
df.columns = ['event','kickoff_time','team_a','team_h','off_a','def_a','off_h','def_h']

In [269]:
team_lst = df['team_a'].unique().tolist()

In [270]:
df['h_score'] = df['off_h'] - df['def_a']
df['a_score'] = df['off_a'] - df['def_h']

In [323]:
fixtures_def = {} #offense faced
for i in team_lst:
    lst = []
    for idx,row in df.iterrows():
        if row['team_a'] == i:
            lst.append(row['off_h'])
        elif row['team_h'] == i:
            lst.append(row['off_a'])
        if len(lst) >9:
            break
    fixtures_def[i] = lst


In [305]:
teams_def = {} #offense faced
for i in team_lst:
    lst = []
    for idx,row in df.iterrows():
        if row['team_a'] == i:
            lst.append(row['team_h'])
        elif row['team_h'] == i:
            lst.append(row['team_a'])
        if len(lst) >9:
            break
    teams_def[i] = lst

In [306]:
off_faced = np.array([v for k,v in fixtures_def.items()])
off_faced_teams = np.array([v for k,v in teams_def.items()])

In [307]:
def get_logo(clubs):
    logo_urls = []
    images = []
    for file in os.listdir(r"C:\Users\limwe\OneDrive\Documents\Computing Projects\Sports Analytics\Data\Logos\Teams\\"):
        images.append(file.replace('.png',''))
    def similar(a,b):
        return SequenceMatcher(None,a,b).ratio()
    similar_dict= dict() 
    for tochange in team_lst:
        most = (0,0)
        for im in images:
            if (unidecode.unidecode(tochange) in unidecode.unidecode(im)) or (unidecode.unidecode(im) in unidecode.unidecode(tochange)):
                most = (1,im)
                break
            tmp = similar(tochange,im)
            if tmp < 0.70:
                continue
            elif tmp > most[0] and im not in similar_dict.values():
                most = (tmp,im)
        similar_dict[tochange] = most[1]
    for club in clubs:
        #name = club.replace('FC','')
        name = unidecode.unidecode(club)
        first = name.split()[0] 
        if os.path.exists(rf'C:\Users\limwe\OneDrive\Documents\Computing Projects\Sports Analytics\Data\Logos\Teams\{name}.png'):
            name = r'C:\Users\limwe\OneDrive\Documents\Computing Projects\Sports Analytics\Data\Logos\Teams\\' + name + '.png'
            logo_urls.append(name)
        elif os.path.exists(rf'C:\Users\limwe\OneDrive\Documents\Computing Projects\Sports Analytics\Data\Logos\Teams\{first}.png'):
            name = r'C:\Users\limwe\OneDrive\Documents\Computing Projects\Sports Analytics\Data\Logos\Teams\\' + first + '.png'
            logo_urls.append(name)
        else: 
            club = similar_dict.get(club)
            name = r'C:\Users\limwe\OneDrive\Documents\Computing Projects\Sports Analytics\Data\Logos\Teams\\' + club +'.png'
            logo_urls.append(name)
        #check
    for url in logo_urls:
        if not os.path.exists(url):
            print('Error:',url)
    return logo_urls

In [ ]:
'''
font = 'Roboto'
title_size = 28
title_color = '#261C15'
subtitle_size = 17
notes_size = 11
notes_color = '#261C15'
bg_color = '#E8B9AB'
axis_color = '#261C15'
bad = '#C33149'
good = '#3E8914'
cmap_lst = [good,'#F7E733',bad]

cmap=LinearSegmentedColormap.from_list('test',cmap_lst, N=256) 
fig,ax  = plt.subplots()
fig.set_size_inches(20,16)
fig.patch.set_facecolor(bg_color)
ax.imshow(off_faced,interpolation='nearest',extent=(0.5,20.5,20,0),cmap=cmap)
logos = get_logo(team_lst)
logos.reverse()
for i in range(20):
    for j in range(10):
        ax.text(2*j+1.5,i+0.5,off_faced_teams[i][j],va='center',ha='center',wrap=True,fontsize=10,font=font,color=notes_color,fontweight='bold')
    imageax = ax.inset_axes([-0.06,(i-1)/20+0.055,0.035,0.035],zorder=2)
    im = plt.imread(logos[i])
    imageax.imshow(im)
    imageax.axis('off')
plt.xticks([i*2-0.5 for i in range(1,11)],['GW '+ str(i) for i in range(1,11)],fontweight='roman',size=13,color=axis_color)
ax.set_yticks([])
ax.tick_params(axis=u'both', which=u'both',length=0)
ax.xaxis.tick_top()
ax.spines['left'].set_color(axis_color)
ax.spines['bottom'].set_color(axis_color)
ax.spines['right'].set_color(axis_color)
ax.spines['top'].set_color(axis_color)
_ = fig.add_artist(lines.Line2D([0.18,0.27],[0.985],linewidth=8,color=title_color))
logoax = fig.add_axes([0.75,0.91,0.07,0.07])
img= Image.open(r"C:\Users\limwe\OneDrive\Documents\Computing Projects\Sports Analytics\Data\Logos\Leagues\Premier League.png").convert('LA')
pixels = img.load()
# for i in range(img.size[0]): # for every pixel:
#     for j in range(img.size[1]):
#         if pixels[i,j][1] != 0:
#             tmp = list(pixels[i,j])
#             tmp[0] = 255
#             pixels[i,j] = tuple(tmp) # change to white
_ = logoax.imshow(img,alpha=0.7)
logoax.axis('off')

fig.text(0.165,0.995,' ')
fig.text(x=0.18,y=0.955,s='Who has the easiest defensive fixture?',ha='left',va='center',font=font,color=title_color,fontsize=title_size,fontweight='semibold')
fig_text(x=0.18,y=0.928,s='How <strong> or <weak> are the attacks faced by teams in the first 10 GWs',highlight_textprops=[{'color':bad},{'color':good}],ha='left',va='center',font=font,color=title_color,fontsize=subtitle_size)


ax = fig.add_axes([0.76,0.1,0.015,0.015])
im = plt.imread(r"C:\Users\limwe\OneDrive\Documents\Computing Projects\Sports Analytics\Visualisations\Assets\icon.png")
_ = fig.text(0.81,0.107,'@nrehiew',ha='right',va='center',font=font,fontsize =notes_size,color = notes_color,alpha=0.8,style ='italic')
ax.imshow(im)
ax.axis('off')
plt.savefig('Completed/defensive.jpg',bbox_inches='tight',facecolor=fig.get_facecolor(), edgecolor='none', dpi=300)'''

In [ ]:
'''
fixtures_off = {} #defense faced
for i in team_lst:
    lst = []
    for idx,row in df.iterrows():
        if row['team_a'] == i:
            lst.append(row['def_h'])
        elif row['team_h'] == i:
            lst.append(row['def_a'])
        if len(lst) >9:
            break
    fixtures_off[i] = lst
teams_off = {} #defemse faced
for i in team_lst:
    lst = []
    for idx,row in df.iterrows():
        if row['team_a'] == i:
            lst.append(row['team_h'])
        elif row['team_h'] == i:
            lst.append(row['team_a'])
        if len(lst) >9:
            break
    teams_off[i] = lst
def_faced = np.array([v for k,v in fixtures_off.items()])
def_faced_teams = np.array([v for k,v in teams_off.items()])
font = 'Roboto'
title_size = 28
title_color = '#261C15'
subtitle_size = 17
notes_size = 11
notes_color = '#261C15'
bg_color = '#E8B9AB'
axis_color = '#261C15'
bad = '#C33149'
good = '#3E8914'
cmap_lst = [bad,'#F7E733',good]

cmap=LinearSegmentedColormap.from_list('test',cmap_lst, N=256) 
fig,ax  = plt.subplots()
fig.set_size_inches(20,16)
fig.patch.set_facecolor(bg_color)
ax.imshow(def_faced,interpolation='nearest',extent=(0.5,20.5,20,0),cmap=cmap)
logos = get_logo(team_lst)
logos.reverse()
for i in range(20):
    for j in range(10):
        ax.text(2*j+1.5,i+0.5,def_faced_teams[i][j],va='center',ha='center',wrap=True,fontsize=10,font=font,color=notes_color,fontweight='bold')
    imageax = ax.inset_axes([-0.06,(i-1)/20+0.055,0.035,0.035],zorder=2)
    im = plt.imread(logos[i])
    imageax.imshow(im)
    imageax.axis('off')
plt.xticks([i*2-0.5 for i in range(1,11)],['GW '+ str(i) for i in range(1,11)],fontweight='roman',size=13,color=axis_color)
ax.set_yticks([])
ax.tick_params(axis=u'both', which=u'both',length=0)
ax.xaxis.tick_top()
ax.spines['left'].set_color(axis_color)
ax.spines['bottom'].set_color(axis_color)
ax.spines['right'].set_color(axis_color)
ax.spines['top'].set_color(axis_color)
_ = fig.add_artist(lines.Line2D([0.18,0.27],[0.985],linewidth=8,color=title_color))
logoax = fig.add_axes([0.75,0.91,0.07,0.07])
img= Image.open(r"C:\Users\limwe\OneDrive\Documents\Computing Projects\Sports Analytics\Data\Logos\Leagues\Premier League.png").convert('LA')
pixels = img.load()
# for i in range(img.size[0]): # for every pixel:
#     for j in range(img.size[1]):
#         if pixels[i,j][1] != 0:
#             tmp = list(pixels[i,j])
#             tmp[0] = 255
#             pixels[i,j] = tuple(tmp) # change to white
_ = logoax.imshow(img,alpha=0.7)
logoax.axis('off')

fig.text(0.165,0.995,' ')
fig.text(x=0.18,y=0.955,s='Who has the easiest offensive fixture?',ha='left',va='center',font=font,color=title_color,fontsize=title_size,fontweight='semibold')
fig_text(x=0.18,y=0.928,s='How <strong> or <weak> are the defenses faced by teams in the first 10 GWs',highlight_textprops=[{'color':bad},{'color':good}],ha='left',va='center',font=font,color=title_color,fontsize=subtitle_size)


ax = fig.add_axes([0.76,0.1,0.015,0.015])
im = plt.imread(r"C:\Users\limwe\OneDrive\Documents\Computing Projects\Sports Analytics\Visualisations\Assets\icon.png")
_ = fig.text(0.81,0.107,'@nrehiew',ha='right',va='center',font=font,fontsize =notes_size,color = notes_color,alpha=0.8,style ='italic')
ax.imshow(im)
ax.axis('off')
plt.savefig('Completed/offensive.jpg',bbox_inches='tight',facecolor=fig.get_facecolor(), edgecolor='none', dpi=300)'''

In [408]:
#offense faced higher ranks means stronger offense
combo = {}
for i in team_lst:
    for j in team_lst:
        if i == j:
            pass
        else:
            i_fix = fixtures_def.get(i)
            j_fix = fixtures_def.get(j)
            lst = [min(x,y) for x,y in zip(i_fix,j_fix)] #will always choose lower ranks (weaker offense)
            combo[i+'/'+j] = lst
def_combo = {i: sum(combo.get(i))/len(combo.get(i)) for i in combo}
def_combo = pd.DataFrame(def_combo.items()).sort_values(1)


In [459]:
font = 'Roboto'
title_size = 28
title_color = '#261C15'
subtitle_size = 17
notes_size = 11
notes_color = '#261C15'
bg_color = '#E8B9AB'
axis_color = '#261C15'
bad = '#C33149'
good = '#3E8914'
cmap_lst = [good,'#F7E733',bad]
not_selected = '#5F758E'
highlight = '#27474E'

In [483]:
def plot_heatmap(team_lst,ax,first,type):
    fixtures_off = {} #defense faced
    for i in team_lst:
        lst = []
        for idx,row in df.iterrows():
            if row['team_a'] == i:
                lst.append(row[f'{type}_h'])
            elif row['team_h'] == i:
                lst.append(row[f'{type}_a'])
            if len(lst) >9:
                break
        fixtures_off[i] = lst
    teams_off = {} #defemse faced
    for i in team_lst:
        lst = []
        for idx,row in df.iterrows():
            if row['team_a'] == i:
                lst.append(row['team_h'])
            elif row['team_h'] == i:
                lst.append(row['team_a'])
            if len(lst) >9:
                break
        teams_off[i] = lst
    def_faced = np.array([v for k,v in fixtures_off.items()])
    def_faced_teams = np.array([v for k,v in teams_off.items()])
    cmap=LinearSegmentedColormap.from_list('test',cmap_lst, N=256) 
    ax.imshow(def_faced,interpolation='nearest',extent=(0.5,20.5,2,0),cmap=cmap)
    logos = get_logo(team_lst)
    logos.reverse()
    for i in range(len(team_lst)):
        for j in range(10):
            if i == 0:
                if def_faced[i][j] > def_faced[i+1][j]:
                    ax.text(2*j+1.5,i+0.5,def_faced_teams[i][j],va='center',ha='center',wrap=True,fontsize=10,font=font,color=notes_color,fontweight='normal')
                else:
                    ax.text(2*j+1.5,i+0.5,def_faced_teams[i][j],va='center',ha='center',wrap=True,fontsize=10,font=font,color='white',fontweight='bold',bbox=dict(boxstyle='round',facecolor=highlight,alpha=0.9,edgecolor=highlight))
            else:
                if def_faced[i][j] > def_faced[i-1][j]:
                    ax.text(2*j+1.5,i+0.5,def_faced_teams[i][j],va='center',ha='center',wrap=True,fontsize=10,font=font,color=notes_color,fontweight='normal')
                else:
                    ax.text(2*j+1.5,i+0.5,def_faced_teams[i][j],va='center',ha='center',wrap=True,fontsize=10,font=font,color='white',fontweight='bold',bbox=dict(boxstyle='round',facecolor=highlight,alpha=0.9,edgecolor=highlight))
        imageax = ax.inset_axes([-0.23,i/2+0.01,0.4,0.4],zorder=2)
        im = plt.imread(logos[i])
        imageax.imshow(im)
        imageax.axis('off')
    if first:
        plt.xticks([i*2-0.5 for i in range(1,11)],['GW '+ str(i) for i in range(1,11)],fontweight='roman',size=13,color=axis_color)
    else:
        ax.set_xticks([])
    ax.set_yticks([])
    ax.tick_params(axis=u'both', which=u'both',length=0)
    ax.xaxis.tick_top()
    ax.spines['left'].set_color(axis_color)
    ax.spines['bottom'].set_color(axis_color)
    ax.spines['right'].set_color(axis_color)
    ax.spines['top'].set_color(axis_color)
    return ax

In [ ]:
fig = plt.figure(tight_layout=True)
gs = gridspec.GridSpec(5,5)
first = True
cmap_lst = [good,'#F7E733',bad]
def_combo = def_combo.sort_values(1)
for i in range(5):
    teams = def_combo.iloc[i*2][0].split('/')
    ax = fig.add_subplot(gs[i,:])
    ax = plot_heatmap(teams,ax,first,'off') #offense faced 
    first = False
    #only for defense
    rank = abs(20-def_combo.iloc[i*2][1])
    s = 'Average Rank of Offense Faced: '+ str(rank)
    ax.text(20.5,2.3,s,ha='right',va='center')
fig.set_size_inches(16,12)
plt.subplots_adjust(wspace=0, hspace=0)
fig.patch.set_facecolor(bg_color)
_ = fig.add_artist(lines.Line2D([0,0.18],[1.07],linewidth=10,color=title_color))
logoax = fig.add_axes([0.9,0.985,0.08,0.08])
img= Image.open(r"C:\Users\limwe\OneDrive\Documents\Computing Projects\Sports Analytics\Data\Logos\Leagues\Premier League.png").convert('LA')
pixels = img.load()
_ = logoax.imshow(img,alpha=0.7)
logoax.axis('off')
ax = fig.add_axes([0.925,0,0.015,0.015])
im = plt.imread(r"C:\Users\limwe\OneDrive\Documents\Computing Projects\Sports Analytics\Visualisations\Assets\icon.png")
_ = fig.text(0.94,0.007,'@nrehiew',ha='left',va='center',font=font,fontsize =notes_size,color = notes_color,alpha=0.8,style ='italic')
ax.imshow(im)
ax.axis('off')
fig.text(-0.02,1.08,' ')
fig.text(1.004,1.08,' ')
fig.text(x=0,y=1.03,s='Best Defensive Combinations',ha='left',va='center',font=font,color=title_color,fontsize=title_size,fontweight='semibold')
fig.text(0,1,'Combinations of teams facing the weakest attacks in the first 10 GWs',ha='left',va='center',font=font,color=title_color,fontsize=subtitle_size)
plt.savefig('Completed/defensive_combo.jpg',bbox_inches='tight',facecolor=fig.get_facecolor(), edgecolor='none', dpi=300)

In [518]:
#offense faced higher ranks means stronger offense
combo = {}
for i in team_lst:
    for j in team_lst:
        if i == j:
            pass
        else:
            i_fix = fixtures_def.get(i)
            j_fix = fixtures_def.get(j)
            lst = [min(x,y) for x,y in zip(i_fix,j_fix)] #will always choose lower ranks (weaker offense)
            combo[i+'/'+j] = lst
def_combo = {i: sum(combo.get(i))/len(combo.get(i)) for i in combo}
def_combo = pd.DataFrame(def_combo.items()).sort_values(1)
lst = []
lst1=  []
lst2=[]
for i,row in def_combo.iterrows():
    counter = 0
    counter1 = 0
    team = row[0].split('/')[0]
    team1 = row[0].split('/')[1]
    fixture = fixtures_def.get(team)
    fixture1 = fixtures_def.get(team1)
    lst.append(np.mean(fixture))
    for x,y in zip(fixture,fixture1):
        if x>y:
            counter +=1
        else:
            counter1 +=1
    lst1.append(counter)
    lst2.append(counter1)
def_combo['team1'] = lst1
def_combo['team2'] = lst2
def_combo[2]= lst
def_combo['diff'] = def_combo[2] - def_combo[1]
def_combo = def_combo.sort_values('diff',ascending=False)
def_combo['balance'] = def_combo['team1'] - def_combo['team2']

In [ ]:
fig = plt.figure(tight_layout=True)
gs = gridspec.GridSpec(5,5)
first = True
cmap_lst = [good,'#F7E733',bad]
balanced = def_combo[def_combo['balance'] <4]
for i in range(5):
    teams = balanced.iloc[i*2][0].split('/')
    ax = fig.add_subplot(gs[i,:])
    ax = plot_heatmap(teams,ax,first,'off') #offense faced 
    first = False
    #only for defense
    rank = abs(balanced.iloc[i*2]['diff'])
    s = 'Change in rank of offense faced due to combination '+ str(round(rank,2))
    ax.text(20.5,2.3,s,ha='right',va='center')
fig.set_size_inches(16,12)
plt.subplots_adjust(wspace=0, hspace=0)
fig.patch.set_facecolor(bg_color)
_ = fig.add_artist(lines.Line2D([0,0.18],[1.07],linewidth=10,color=title_color))
logoax = fig.add_axes([0.9,0.985,0.08,0.08])
img= Image.open(r"C:\Users\limwe\OneDrive\Documents\Computing Projects\Sports Analytics\Data\Logos\Leagues\Premier League.png").convert('LA')
pixels = img.load()
_ = logoax.imshow(img,alpha=0.7)
logoax.axis('off')
ax = fig.add_axes([0.925,0,0.015,0.015])
im = plt.imread(r"C:\Users\limwe\OneDrive\Documents\Computing Projects\Sports Analytics\Visualisations\Assets\icon.png")
_ = fig.text(0.94,0.007,'@nrehiew',ha='left',va='center',font=font,fontsize =notes_size,color = notes_color,alpha=0.8,style ='italic')
_ = fig.text(0,0.007,'Minimum 4 fixtures per team to be displayed here',ha='left',va='center',font=font,fontsize =notes_size,color = notes_color,alpha=0.8,style ='italic')
ax.imshow(im)
ax.axis('off')
fig.text(-0.02,1.08,' ')
fig.text(1.004,1.08,' ')
fig.text(x=0,y=1.03,s='Best Defensive Combinations',ha='left',va='center',font=font,color=title_color,fontsize=title_size,fontweight='semibold')
fig.text(0,1,'Combinations of teams which improves their collective fixture run',ha='left',va='center',font=font,color=title_color,fontsize=subtitle_size)
plt.savefig('Completed/defensive_combo_balanced.jpg',bbox_inches='tight',facecolor=fig.get_facecolor(), edgecolor='none', dpi=300)

In [ ]:
#offense faced higher ranks means stronger offense
combo = {}
for i in team_lst:
    for j in team_lst:
        if i == j:
            pass
        else:
            i_fix = fixtures_def.get(i)
            j_fix = fixtures_def.get(j)
            lst = [min(x,y) for x,y in zip(i_fix,j_fix)] #will always choose lower ranks (weaker offense)
            combo[i+'/'+j] = lst
def_combo = {i: sum(combo.get(i))/len(combo.get(i)) for i in combo}
def_combo = pd.DataFrame(def_combo.items()).sort_values(1)
lst = []
lst1=  []
lst2=[]
for i,row in def_combo.iterrows():
    counter = 0
    counter1 = 0
    team = row[0].split('/')[0]
    team1 = row[0].split('/')[1]
    fixture = fixtures_def.get(team)
    fixture1 = fixtures_def.get(team1)
    lst.append(np.mean(fixture))
    for x,y in zip(fixture,fixture1):
        if x>y:
            counter +=1
        else:
            counter1 +=1
    lst1.append(counter)
    lst2.append(counter1)
def_combo['team1'] = lst1
def_combo['team2'] = lst2
def_combo[2]= lst
def_combo['diff'] = def_combo[2] - def_combo[1]
def_combo = def_combo.sort_values('diff',ascending=False)
def_combo['balance'] = def_combo['team1'] - def_combo['team2']

In [ ]:
fig = plt.figure(tight_layout=True)
gs = gridspec.GridSpec(5,5)
first = True
cmap_lst = [good,'#F7E733',bad]
def_combo = def_combo.sort_values(1)
for i in range(5):
    teams = def_combo.iloc[i*2][0].split('/')
    ax = fig.add_subplot(gs[i,:])
    ax = plot_heatmap(teams,ax,first,'off') #offense faced 
    first = False
    #only for defense
    rank = abs(20-def_combo.iloc[i*2][1])
    s = 'Average Rank of Offense Faced: '+ str(rank)
    ax.text(20.5,2.3,s,ha='right',va='center')
fig.set_size_inches(16,12)
plt.subplots_adjust(wspace=0, hspace=0)
fig.patch.set_facecolor(bg_color)
_ = fig.add_artist(lines.Line2D([0,0.18],[1.07],linewidth=10,color=title_color))
logoax = fig.add_axes([0.9,0.985,0.08,0.08])
img= Image.open(r"C:\Users\limwe\OneDrive\Documents\Computing Projects\Sports Analytics\Data\Logos\Leagues\Premier League.png").convert('LA')
pixels = img.load()
_ = logoax.imshow(img,alpha=0.7)
logoax.axis('off')
ax = fig.add_axes([0.925,0,0.015,0.015])
im = plt.imread(r"C:\Users\limwe\OneDrive\Documents\Computing Projects\Sports Analytics\Visualisations\Assets\icon.png")
_ = fig.text(0.94,0.007,'@nrehiew',ha='left',va='center',font=font,fontsize =notes_size,color = notes_color,alpha=0.8,style ='italic')
ax.imshow(im)
ax.axis('off')
fig.text(-0.02,1.08,' ')
fig.text(1.004,1.08,' ')
fig.text(x=0,y=1.03,s='Best Defensive Combinations',ha='left',va='center',font=font,color=title_color,fontsize=title_size,fontweight='semibold')
fig.text(0,1,'Combinations of teams facing the weakest attacks in the first 10 GWs',ha='left',va='center',font=font,color=title_color,fontsize=subtitle_size)
# plt.savefig('Completed/offensive_combo.jpg',bbox_inches='tight',facecolor=fig.get_facecolor(), edgecolor='none', dpi=300)